In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sidetable
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

In [5]:
df_da = pd.read_csv('data/df_da.csv', index_col = 'index')
pd.options.display.max_columns = None
df_da.head(2)

,time,age,gender,country,education,role,coding_experience,prog_language,code_env,notebook_kaggle,notebook_colab,notebook_azure,notebook_paperspace,notebook_binder,notebook_codeocean,notebook_ibmwatsonstudio,notebook_amazonsagemaker,notebook_amazonemr,notebook_googlecloudnotebooks,notebook_googleclouddatalab,notebook_databricks,notebook_zeppelin,notebook_deepnote,notebook_observable,notebook_OTHER,viz,field,salary,learn_dp_MySQL,learn_dp_PostgreSQL,learn_dp_SQLite,learn_dp_Oracle,learn_dp_Mongo,learn_dp_Snowflake,learn_dp_IBMDd2,learn_dp_SQLServer,learn_dp_AzureSQL,learn_dp_AzureCosmos,learn_dp_Redshift,learn_dp_Aurora,learn_dp_RDS,learn_dp_DynamoDB,learn_dp_BigQuery,learn_dp_CloudSQL,learn_dp_Firestore,learn_dp_BigTable,learn_dp_Spanner,learn_dp_Other,dp_most_freq,BI_tools,share_plotlydash,share_streamlit,share_NBViewer,share_GitHub,share_Personal blog,share_Kaggle,share_Colab,share_Shiny,share_None,share_OTHER,courses_Coursera,courses_edX,courses_Kaggle,courses_DataCamp,courses_Fastai,courses_Udacity,courses_Udemy,courses_LinkedIn,courses_Cloud-certification,courses_University,courses_OTHER,media_Twitter,media_Email,media_Reddit,media_Kaggle,media_Forums,media_YouTube,media_Podcasts,media_Blogs,media_Journal,media_Slack,media_OTHER,cat_experience,cat_salary
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
16,1315,50-54,Man,Belgium,Bachelor’s degree,Data Analyst,20+ years,"Python, SQL","Jupyter (JupyterLab, Jupyter Notebooks, etc) ,...",Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Databricks Collaborative Notebooks,NaN,NaN,NaN,NaN,"Matplotlib ,Seaborn ,Plotly / Plotly Express",Energy/Mining,"2,000-2,999",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Microsoft Azure SQL Database,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Coursera,NaN,Kaggle Learn Courses,NaN,NaN,NaN,NaN,NaN,"Cloud-certification programs (direct from AWS,...",NaN,NaN,NaN,NaN,NaN,"Kaggle (notebooks, forums, etc)",NaN,NaN,NaN,"Blogs (Towards Data Science, Analytics Vidhya,...",NaN,NaN,NaN,Senior+,< 25K
32,171,22-24,Nonbinary,United States of America,Some college/university study without earning ...,Data Analyst,< 1 years,R,RStudio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Cloud Datalab,NaN,NaN,NaN,NaN,NaN,Ggplot / ggplot2,Manufacturing/Fabrication,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Junior,NaN


In [9]:
df_da.shape

(3269, 84)

In [11]:
df_da.dtypes

time               int64
age               object
gender            object
country           object
education         object
                   ...  
media_Journal     object
media_Slack       object
media_OTHER       object
cat_experience    object
cat_salary        object
Length: 84, dtype: object

In [6]:
df_da.describe(include='object').T

,count,unique,top,freq
age,3269,11,25-29,788
gender,3269,5,Man,2538
country,3269,66,India,795
education,3269,7,Master’s degree,1514
role,3269,2,Data Analyst,2301
...,...,...,...,...
media_Journal,260,1,"Journal Publications (peer-reviewed journals, ...",260
media_Slack,258,1,"Slack Communities (ods.ai, kagglenoobs, etc)",258
media_OTHER,82,1,Other,82
cat_experience,3269,5,Junior,1881


In [10]:
df_da.describe(include=np.number).T

,count,mean,std,min,25%,50%,75%,max
time,3269.0,11948.610278,104455.828821,128.0,478.0,696.0,1119.0,2414534.0
